In [51]:
import pandas as pd
import numpy as np

from utils import temporal_from_strings
from temporal_mining import augment_by_column
from datasets import generate_dataset_from_user
from data_classes import PMDataFitbitDataset

# Generate augmented dataset

In [52]:
user_index = 1
temporal_window = 3
dataset = generate_dataset_from_user(PMDataFitbitDataset(), user_index=user_index)
temporal_dataset = temporal_from_strings(dataset)
augmented_dataset = augment_by_column(temporal_dataset, temporal_window=temporal_window)

c:\Users\the_h\Desktop\POSTDOC\FITBIT\CODE\aged_apriori\datasets.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_quality_df['timestamp'] = sleep_quality_df['timestamp'].astype('datetime64')          # change type to allow later merge


# Convert ALBA dataset to GSP notation

In [53]:
ALBA_to_GSP_dict = {}
GSP_to_ALBA_dict = {}
seq_n = 0
GSP_dataset = []
sep = True
for sequence in augmented_dataset:
    GSP_sequence = []
    previous_item_timeloc = sequence[0].timeloc
    for item in sequence:
        if previous_item_timeloc != item.timeloc and sep:
            previous_item_timeloc = item.timeloc
            GSP_sequence.append(-1)
        if item not in ALBA_to_GSP_dict:
            ALBA_to_GSP_dict[item] = seq_n
            GSP_to_ALBA_dict[seq_n] = item
            GSP_sequence.append(seq_n)
            seq_n += 1
        else:
            GSP_sequence.append(ALBA_to_GSP_dict[item])
    GSP_sequence.append(-1)
    GSP_sequence.append(-2)
    GSP_dataset.append(GSP_sequence)

text_dataset = []
for sequence in GSP_dataset:
    line = ''
    for item in sequence:
        line += str(item) + ' '
    text_dataset.append(line.strip() + '\n')



In [54]:
#maybe we could also write the GSP_to_ALBA_dict to a file and load it later
with open('for_GSP.txt', 'w+') as f:
    for line in text_dataset:
        f.writelines(line)

# Convert GSP results to ALBA notation

In [85]:
results = []
with open('res.txt', 'r') as f:
    results = f.readlines()


In [87]:
def res_line_to_GSP_seq(line: str) -> list[int]:
    seq = line.split('#')[0]
    int_seq = [int(x) for x in seq.strip().split(' ') if x != '-1']
    supp = int(line.split('#')[1].strip().split(':')[1].strip())
    return int_seq, supp

ALBA_res = []
supp_list = []
for line in results:
    GSP_seq, supp = res_line_to_GSP_seq(line)
    ALBA_seq = []
    for GSP_item in GSP_seq:
        ALBA_item = GSP_to_ALBA_dict[GSP_item]
        ALBA_seq.append(ALBA_item)
    ALBA_res.append(ALBA_seq)
    supp_list.append(supp)


In [89]:
for i in range(10):
    print(ALBA_res[i], ': ', supp_list[i])

[LA_2_t0] :  46
[MA_2_t0] :  46
[HA_1_t0] :  46
[R_2_t0] :  46
[ZL_3_t0] :  46
[LA_2_t1] :  46
[MA_2_t1] :  46
[HA_1_t1] :  46
[R_2_t1] :  46
[ZL_3_t1] :  45
